In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from data_changed import * # 里头的items是我们用来训练的
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import random
import joblib
from torch.utils.data import Dataset

In [2]:
pwd

'/home/douhuanmin/zhangyi_homework/final'

In [3]:
items_mean = items.mean(dim=0)
items_std = items.std(dim=0)
items_normalized = (items - items_mean)/items_std
items_normalized
x = items_normalized[:, :-1]  # 特征张量
y = items_normalized[:, -1]   # 标签张量
train_size = int(0.8 * len(items_normalized))
train_x, test_x = x[:train_size], x[train_size:]
train_y, test_y = y[:train_size], y[train_size:]
scaler = (items_mean, items_std)
joblib.dump(scaler , "diamond_price_scaler.pkl")
train_x.size() , test_x.size() , train_y.size()

(torch.Size([43136, 12]), torch.Size([10784, 12]), torch.Size([43136]))

In [4]:
class DiamondPricePredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim).double()  # 将权重张量和偏置张量的数据类型改为torch.float64
        self.fc2 = nn.Linear(hidden_dim , 512).double()
        self.fc3 = nn.Linear(512 , hidden_dim).double()
        self.fc4 = nn.Linear(hidden_dim ,output_dim).double()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# 初始化模型
input_dim = 12
hidden_dim = 64
output_dim = 1
model = DiamondPricePredictor(input_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(train_x)
    loss = criterion(outputs, train_y.view(-1, 1))
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 打印损失值
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# 在测试集上评估模型
with torch.no_grad():
    test_outputs = model(test_x)
    test_loss = criterion(test_outputs, test_y.view(-1, 1))
    print('Test Loss: {:.4f}'.format(test_loss.item()))
    


Epoch [100/1000], Loss: 0.1154
Epoch [200/1000], Loss: 0.1072
Epoch [300/1000], Loss: 0.1054
Epoch [400/1000], Loss: 0.1045
Epoch [500/1000], Loss: 0.1038
Epoch [600/1000], Loss: 0.1033
Epoch [700/1000], Loss: 0.1029
Epoch [800/1000], Loss: 0.1025
Epoch [900/1000], Loss: 0.1022
Epoch [1000/1000], Loss: 0.1021
Test Loss: 0.0158


In [5]:
torch.save(model, 'diamond_price.pt')
from data_need_pre import *
with torch.no_grad():
    test_outputs = model(test_x)
    test_loss = criterion(test_outputs, test_y.view(-1, 1))
    print('Test Loss: {:.4f}'.format(test_loss.item()))

Test Loss: 0.0158


In [6]:

# 加载模型和归一化器
model = torch.load('diamond_price.pt')
scaler = joblib.load('diamond_price_scaler.pkl')

# 准备输入数据
input_data = data_need_pred

# 计算输入数据的均值和标准差
input_data_mean = input_data.mean(dim=0, keepdim=True)
input_data_std = input_data.std(dim=0, keepdim=True)

# 对输入数据进行归一化处理
input_data_normalized = (input_data - input_data_mean) / input_data_std

# 将输入数据的数据类型转换为与模型权重相同的数据类型
input_data_normalized = input_data_normalized.to(model.fc1.weight.dtype)

# 预测归一化的价格
predicted_price_normalized = model(input_data_normalized)

# 反归一化输出价格
predicted_price = predicted_price_normalized * scaler[1][-1] + scaler[0][-1]

# 打印预测的价格
predicted_price_np = predicted_price.detach().numpy()
for i in range(len(predicted_price_np)):
    print('Predicted price {}: ${:.2f}'.format(i+1, float(predicted_price_np[i])))
print(type(predicted_price_np))
predicted_price_np = predicted_price_np.sum(axis=1)
predicted_price_np = predicted_price_np.tolist()
print(predicted_price_np)

Predicted price 1: $770.92
Predicted price 2: $38953.07
Predicted price 3: $42561.06
Predicted price 4: $1000.56
Predicted price 5: $76711.12
Predicted price 6: $31357.99
Predicted price 7: $34515.51
Predicted price 8: $40731.79
Predicted price 9: $73338.68
Predicted price 10: $49776.88
Predicted price 11: $4300.86
Predicted price 12: $4692.57
Predicted price 13: $4366.44
Predicted price 14: $6274.51
Predicted price 15: $8495.88
Predicted price 16: $21949.81
Predicted price 17: $8314.60
Predicted price 18: $20644.83
Predicted price 19: $41882.95
Predicted price 20: $32875.65
<class 'numpy.ndarray'>
[770.9192896448731, 38953.06890561105, 42561.055230242, 1000.5572855417377, 76711.11654003663, 31357.985027028826, 34515.512365819384, 40731.79168350308, 73338.67692321925, 49776.876216089455, 4300.864309264187, 4692.567285986468, 4366.435578822737, 6274.512854956334, 8495.880988131215, 21949.809337279723, 8314.599305207896, 20644.82655849697, 41882.95285497815, 32875.64617205613]


In [7]:
predicted_price_np = predicted_price_np
predicted_price_np_rounded = [round(x) for x in predicted_price_np]

f['price'] = {k:predicted_price_np[k] for k in range(len(f['price']))}
# 将数据框写入到Excel文件中（使用XlsxWriter作为引擎）
writer = pd.ExcelWriter('输出.xlsx', engine='xlsxwriter')
f.to_excel(writer, index=False, sheet_name='Sheet1')

# 获取workbook对象
workbook = writer.book

# 关闭Excel写入对象并保存文件
writer.close()
workbook.close()

/home/douhuanmin/anaconda3/envs/env/lib/python3.10/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [8]:
type(predicted_price_np)

list

In [9]:
print(model.state_dict())

OrderedDict([('fc1.weight', tensor([[ 2.4247e-01, -6.2386e-02,  4.4015e-03, -2.7621e-02, -4.1399e-02,
         -7.8320e-02, -4.5458e-03, -4.6089e-02,  8.1657e-02,  1.1223e-01,
         -2.4927e-02, -6.9187e-02],
        [ 1.9825e-01,  6.9036e-03, -3.1057e-02,  1.4953e-01, -2.7676e-02,
         -6.9599e-02,  2.8573e-02,  1.0899e-01,  1.3087e-01,  1.9208e-03,
         -6.2994e-05,  1.1762e-01],
        [-1.9608e-01,  1.4459e-01,  2.0527e-01,  1.7390e-01, -6.0174e-02,
         -3.6162e-02,  6.2043e-02, -1.7717e-02, -6.3689e-02,  9.5190e-02,
          4.2009e-02,  7.0004e-02],
        [-2.1911e-01,  1.6624e-01, -1.2274e-01, -1.1540e-01, -5.0218e-02,
          5.5213e-02, -1.0333e-01, -7.4219e-02,  1.7399e-01,  2.1846e-02,
         -9.9396e-02,  2.3390e-01],
        [-1.5028e-01, -1.2097e-01,  6.9753e-02, -7.3414e-02, -8.5725e-02,
          1.1543e-01,  7.0911e-03, -6.1226e-02,  1.4229e-01, -6.3534e-02,
         -2.5177e-02, -3.4797e-02],
        [-3.7325e-02,  8.1390e-02, -1.5626e-01, -4.1